In [ ]:
#Installing all the packages
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import pandas as pd
import re

!pip install spatial
!pip install wikipedia2vec
import spatial
from scipy.spatial import distance

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import sys

In [ ]:
!wget -P /root/input/ -c "http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_100d.pkl.bz2"

--2020-09-25 16:37:19--  http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_100d.pkl.bz2
Resolving wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)... 52.219.16.13
Connecting to wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)|52.219.16.13|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!bunzip2 "/root/input/enwiki_20180420_100d.pkl.bz2" 

bunzip2: Output file /root/input/enwiki_20180420_100d.pkl already exists.


In [ ]:
from wikipedia2vec import Wikipedia2Vec
wiki2vec=Wikipedia2Vec.load("/root/input/enwiki_20180420_100d.pkl")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving seed23.txt to seed23 (2).txt


In [ ]:
file_name = "seed23.txt"
uploaded[file_name].decode("utf-8")

'problem\r\npoor\r\nterrible\r\nunresponsive\r\ndiminished\r\nminiscule\r\ngem\r\nrude\r\noutrageous\r\nterrific\r\nunfriendly\r\nterrified\r\nscowled\r\nyelled\r\ndispleased\r\ntempered\r\nunhappy\r\ndirty\r\ndreary\r\naccused\r\nhumiliated\r\nvaguest\r\nridiculously\r\nugly\r\noverpriced\r\nrecession\r\nstressful\r\noutrageously\r\npoor\r\nmiserable\r\ncranky\r\nunappetizing\r\ndisappointed\r\nblindfolded\r\ndisgusted\r\nwarning\r\nsnubbed\r\ninept\r\nrepulsive\r\narrogant\r\ncallous\r\nawkward\r\nexcellent\r\nrelaxed\r\nwonderful\r\nheartbeat\r\nbest\r\nmassive\r\nelite\r\nclassy\r\nelegant\r\noutstanding\r\nmagnificent\r\nhomey\r\naffordable\r\nequipped\r\nextravagant\r\ncozy\r\nfriendly\r\nprofessional\r\ngreat\r\nfantastic\r\ndelicious\r\nincredibly\r\nbeauty\r\nrelieved\r\ncomplimentary\r\npriceless\r\ndefinitely\r\nrecommended\r\ngorgeous\r\nguarantees\r\ncelebrity\r\npleased\r\nexceptional\r\nsuggest'

In [ ]:
seed = uploaded[file_name].decode("utf-8").split("\r\n")
seed = [[i for i in seed]]

In [ ]:
# print(len(seed[0]))

76
[['problem', 'poor', 'terrible', 'unresponsive', 'diminished', 'miniscule', 'gem', 'rude', 'outrageous', 'terrific', 'unfriendly', 'terrified', 'scowled', 'yelled', 'displeased', 'tempered', 'unhappy', 'dirty', 'dreary', 'accused', 'humiliated', 'vaguest', 'ridiculously', 'ugly', 'overpriced', 'recession', 'stressful', 'outrageously', 'poor', 'miserable', 'cranky', 'unappetizing', 'disappointed', 'blindfolded', 'disgusted', 'warning', 'snubbed', 'inept', 'repulsive', 'arrogant', 'callous', 'awkward', 'excellent', 'relaxed', 'wonderful', 'heartbeat', 'best', 'massive', 'elite', 'classy', 'elegant', 'outstanding', 'magnificent', 'homey', 'affordable', 'equipped', 'extravagant', 'cozy', 'friendly', 'professional', 'great', 'fantastic', 'delicious', 'incredibly', 'beauty', 'relieved', 'complimentary', 'priceless', 'definitely', 'recommended', 'gorgeous', 'guarantees', 'celebrity', 'pleased', 'exceptional', 'suggest']]


In [ ]:
#importing the dataset
from google.colab import files
uploaded = files.upload()

Saving Labelleddataset.csv to Labelleddataset (3).csv


In [ ]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Labelleddataset.csv']))

In [ ]:
x=df2['text']
y=df2['decision']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
print(x_train.head())
print(y_train.head())

222     WOW!! Where do I start? Booked this hotel thro...
204     The Hard Rock Hotel was a natural choice for m...
792     Extravagant, Exuberant Experience! Our stay at...
510     The Sheraton Chicago Hotel and Towers is a mag...
1498    I was initially excited when I found out that ...
Name: text, dtype: object
222      truthful
204      truthful
792     deceptive
510     deceptive
1498    deceptive
Name: decision, dtype: object


In [ ]:
count= 0
reviews = [[i] for i in x_train]
print("Number of reviews=",len(reviews))

Number of reviews= 1280


In [ ]:
texts = []

for review in reviews:
  temp = re.sub(r'\[[0-9]*\]',' ',review[0])
  temp = re.sub(r'\s+',' ',temp)
  temp = temp.lower()
  temp = re.sub(r'\d',' ',temp)
  texts.append(re.sub(r'\s+',' ',temp)) 


In [ ]:
sentences=[]
for text in texts:
  sentence = nltk.sent_tokenize(text)
  sentences.append(sentence)



In [ ]:
word_tokenized_reviews = []

for review in sentences:
  temp = [nltk.word_tokenize(sentence) for sentence in review]
  word_tokenized_reviews.append(temp)


In [ ]:
#Removing all the stopwords and the punctuations
for review in word_tokenized_reviews:   # the review will contain [[sentence],[sentence]]
  for i in range(len(review)):
    review[i] = [word for word in review[i] if word not in stopwords.words('english')]  #represents each sentence
    table = str.maketrans("","",string.punctuation)
    stripped = [w.translate(table) for w in review[i]]
    review[i] = list(filter(('').__ne__, stripped))
    # print(review[i])

In [ ]:
# for each review in word_tokenized_reviews
count=0
additional_words=[]
for it in range(0,1280):#for each review
  print("Iteration="+str(it))
  selected_review = word_tokenized_reviews[it]  #selecting the review to be operated upon
  # print("Selected Review=",selected_review)
  candidate_spam_dictionary = []  #intermediate dictionary
  for i in seed[0]: #for each word in seed spam dictionary
    # print("Word selected ="+i)
    dissimilarity_weights = []  #for finding the dissimilarity between the word in seed spam dictionary and the words in the reviews
    try:
      v_spam = wiki2vec.get_word_vector(i)  #finding vector of each seed spam word
      # print("vspam=",v_spam)
    except Exception as e:
      # print("Spam Exception=",e)
      continue
    for j in range(len(selected_review)):  #for each sentence in selected review
      # print(selected_review[j])
      for k in selected_review[j]:         #for each word in the sentence
        # print(k)
        try:
          v_review = wiki2vec.get_word_vector(k)  #converting each word of the sentence into vector
        except Exception as e:
          continue
        # print("v_review=",v_review)
        dissim=distance.cosine(v_spam,v_review) #calculating the cosine distance
        # print(i,k,"=>",dissim)
        if(1-dissim>=0.5):  #similarity
          if [i,k,dissim] not in dissimilarity_weights:
            dissimilarity_weights.append([i,k,dissim])   #dissimilarity weights for each review word
    # print("Dissimilarity=",dissimilarity_weights)
    s = sorted(dissimilarity_weights, key = lambda x: (x[2]))   #sorting the dissimilarity weights of the words from seed words for the entire review
    # print("s=>",s)
    s_dash = []
    if len(s)>=15: #What if there are less than 15 words
      for j in range(0,15): #appending 15 most similar words to the s_dash
        s_dash.append(s[j])
    else:
      for j in range(len(s)):
        s_dash.append(s[j])
    # print("S_dash_length=",len(s_dash))
    # print("s_dash",s_dash)
    for j in range(len(s_dash)): #for each word in s_dash
      if(s_dash[j][1] not in seed[0]):# if s_dash is already present in the dictionary or similarity < 0.4 discard it 1-s_dash becoz its dissimilarity
        if s_dash[j][1] not in candidate_spam_dictionary:
          candidate_spam_dictionary.append(s_dash[j][1])
  # print("Candidate Spam dictionary:",candidate_spam_dictionary)  #No bug in Algorithm for Candidate spam dictionary

  for i in candidate_spam_dictionary:  #for each word in candidate spam dictionary
    # print("Candidate word selected=",i)
    dissimilarity_weights   = []
    v_candidate_dictionary  = wiki2vec.get_word_vector(i)  #vector of each candidate spam word because the candidate spam words has been extracted from the reviews
    for j in range(len(selected_review)):  #for each sentence in the selected review
      # print(selected_review[j])
      for k in selected_review[j]: #for each word in selected sentence
        try:
          v_review = wiki2vec.get_word_vector(k)  #converting each word of the sentence into vector
        except:
          continue
        dissim=distance.cosine(v_candidate_dictionary,v_review) #calculating the dissimilarity
        if [i,k,dissim] not in dissimilarity_weights: # What if the word is already there
          dissimilarity_weights.append([i,k,dissim])   #dissimilarity weights for each review word
    # print("dissimilarity_weights=",dissimilarity_weights)
    s = sorted(dissimilarity_weights, key = lambda x: (x[2]))  #sorting the dissimilarity weights of the words from seed words for the entire review
    # print("s=",s)
    s_double_dash = []
    if(len(s)>=15):
      for j in range(0,15): #Appending 15 most similar words
        s_double_dash.append(s[j][1]) # s_double_dash and s will be three dimensional array [k],storing only append 15 most similar words to s_dash
    else:
      for j in range(len(s)):
        s_double_dash.append(s[j][1])
    # print("s_double_dash=",s_double_dash) #printing the 15 words similar to the canidate spam dictionary
    if (len(set(s_double_dash) & set(seed[0]))>=3):#if there are 3 or more than 3 words in the seed dictionary then add the word of candidate spam dictionary to the seed spam dictionary
      # print("Len=",set(s_double_dash) & set(seed[0]))
      if i not in additional_words:
        # seed[0].append(i)
        additional_words.append(i)
        print("Word included=",i)
  # #       # system.exit(1)      


Iteration=0
Iteration=1
Iteration=2
Word included= well
Word included= much
Word included= enjoyed
Word included= exuberant
Word included= enjoyable
Word included= intoxicating
Word included= highly
Word included= made
Word included= decorative
Iteration=3
Word included= phenomenal
Word included= majestic
Word included= big
Word included= night
Word included= lights
Iteration=4
Iteration=5
Iteration=6
Iteration=7
Word included= superb
Word included= nice
Word included= book
Iteration=8
Iteration=9
Iteration=10
Iteration=11
Iteration=12
Iteration=13
Word included= worse
Word included= tired
Word included= desperate
Word included= stale
Iteration=14
Iteration=15
Iteration=16
Word included= charm
Word included= close
Iteration=17
Word included= even
Word included= thought
Word included= could
Iteration=18
Iteration=19
Iteration=20
Iteration=21
Iteration=22
Word included= experience
Word included= comfortable
Word included= favorite
Iteration=23
Iteration=24
Word included= amazing
Word inc

In [ ]:
print(len(additional_words))

451


In [ ]:
additional_words=[word for word in additional_words if word not in stopwords.words('english')]


In [ ]:
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize 
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:

# print(additional_words)
# print(len(additional_words))
tagged_words = nltk.pos_tag(additional_words)
Final_words = []

print("Before = ",len(additional_words)+len(seed[0]))


# CC	coordinating conjunction
# DT	determiner
# EX	existential there
# IN	preposition/subordinating conjunction
# MD	modal (could, will)
# NNS	noun plural (desks)
# NNP	proper noun, singular (sarah)
# NNPS	proper noun, plural (indians or americans)
# PDT	predeterminer (all, both, half)
# POS	possessive ending (parent\ 's)
# PRP	personal pronoun (hers, herself, him,himself)
# PRP$	possessive pronoun (her, his, mine, my, our )
# RP	particle (about)
# TO	infinite marker (to)
# VB	verb (ask)
# VBG	verb gerund (judging)
# VBD	verb past tense (pleaded)
# VBN	verb past participle (reunified)
# VBP	verb, present tense not 3rd person singular(wrap)
# VBZ	verb, present tense with 3rd person singular (bases)
# WDT	wh-determiner (that, what)
# WP	wh- pronoun (who)
# WRB	wh- adverb (how)


for word,tag in tagged_words:
  if tag not in ["NNS","NNP","NNPS","CC","IN","EX","CC","DT","EX","MD","PDT","POS","PRP","PRP$","VB","WDT","WP","WRB","VBG","VBD","VBN","VBP","VBZ"]:
    Final_words.append(word)
  # else:
  #   print("Word = ",word," Tag = ",tag)

for i in seed[0]:
  Final_words.append(i)
print("After = ",len(Final_words))

Before =  527
After =  407


In [ ]:
f= open("newSeed0.4.txt","w+")

In [ ]:
for i in range(len(Final_words)):
     f.write(Final_words[i]+"\r\n")

f.close()

**For Accuracy Check**

In [ ]:
from google.colab import files # Upload the Dictionary File 
uploaded = files.upload()

Saving newSeed0.4 (4).txt to newSeed0.4 (4).txt


In [ ]:
file_name = "newSeed0.4 (4).txt"  # Adjust the File Names According to the spam Dictionary used
uploaded[file_name].decode("utf-8")

'well\r\nmuch\r\nexuberant\r\nenjoyable\r\nhighly\r\ndecorative\r\nphenomenal\r\nmajestic\r\nbig\r\nnight\r\nsuperb\r\nnice\r\nbook\r\nworse\r\ntired\r\ndesperate\r\nstale\r\ncharm\r\nclose\r\neven\r\ncomfortable\r\nfavorite\r\namazing\r\nperfect\r\nabsolutely\r\nwatch\r\ndecent\r\nbad\r\nmediocre\r\nquiet\r\ngold\r\navoid\r\nnever\r\ngarish\r\nstayed\r\nhusband\r\ndecor\r\nroom\r\nstaff\r\nweekend\r\nstar\r\nbeautiful\r\nhuge\r\nwonderfully\r\nmusic\r\nenough\r\npopular\r\nlittle\r\nperfection\r\nwarm\r\ninsincere\r\nhardly\r\ngood\r\neveryone\r\nlove\r\nyes\r\nclean\r\nwindy\r\nzest\r\nunhelpful\r\nuncomfortable\r\ntruly\r\ntotally\r\nhope\r\npromise\r\nsure\r\nhorrible\r\nfresh\r\nbreathtaking\r\nstrongest\r\nill\r\nexpensive\r\nstiff\r\nattitude\r\nlot\r\nwedding\r\nson\r\nbellhop\r\nbreakable\r\nchoice\r\nfabulous\r\nsexual\r\nspectacular\r\nhelpful\r\nlook\r\ntrue\r\nprecise\r\nrestaurant\r\nnegative\r\nexceptionally\r\nincredible\r\nfun\r\nluxurious\r\nboutique\r\nfamous\r\nmany

In [ ]:
seed = uploaded[file_name].decode("utf-8").split("\r\n")
seed = [[i for i in seed]]
# for i in seed:
#   print(i)

In [ ]:
count= 0
reviews = [[i] for i in x_test]
print("Number of reviews=",len(reviews))

Number of reviews= 320


In [ ]:
print(len(reviews))

320


In [ ]:
texts = []

for review in reviews:
  temp = re.sub(r'\[[0-9]*\]',' ',review[0])
  temp = re.sub(r'\s+',' ',temp)
  temp = temp.lower()
  temp = re.sub(r'\d',' ',temp)
  texts.append(re.sub(r'\s+',' ',temp)) 


print(len(texts))


320


In [ ]:
sentences=[]
for text in texts:
  sentence = nltk.sent_tokenize(text)
  sentences.append(sentence)

print(len(sentences))

320


In [ ]:
word_tokenized_reviews = []

for review in sentences:
  temp = [nltk.word_tokenize(sentence) for sentence in review]
  word_tokenized_reviews.append(temp)

In [ ]:
for review in word_tokenized_reviews:   # the review will contain [[sentence],[sentence]]
  for i in range(len(review)):
    review[i] = [word for word in review[i] if word not in stopwords.words('english')]

In [ ]:
#breaking each review in words
# print(word_tokenized_reviews[0])
review = []
for it in word_tokenized_reviews:
  words_in_each_review=[]
  for i in it: #for each sentence
    for j in i: #for each word
      words_in_each_review.append(j)
  review.append(words_in_each_review)

In [ ]:
print(len(review))

320


In [ ]:
spam_word_count=[]
for i in review:
  # print("set(i)=",set(i))
  # print("set(review)=",set(i))
  # print(len(set(i) & set(seed[0])))
  spam_word_count.append(len(set(i) & set(seed[0])))
  # print(set(i)&set(seed[0]))
  
print(len(spam_word_count))

320


In [ ]:
test=[]
for i in y_test:
  test.append(i)
print(test[1])

truthful


In [ ]:
print(len(spam_word_count))

320


Confusion Matrix

In [ ]:
counting = 0
a=0  # Real spam and predicted spam
b=0  # Real spam but not predicted as spam
c=0  # Not spam but predicted as spam
d=0  # Not spam and not predicted as spam
# print(df2['decision'][0])

for i in range(len(spam_word_count)):
  if spam_word_count[i]>=10: # predicted spam
    if(test[i]=="deceptive"): #actual spam
      a+=1
      # print(df2['decision'][i],"= predicted deceptive",spam_word_count[i])
    else: #actual not spam
      c+=1
      # print(df2['decision'][i],"= predicted deceptive")
  else:  #predicted not spam
    if(test[i]=="truthful"): #actual not spam
      d+=1
      # print(df2['decision'][i],"= predicted truthful")
    else: #actual spam
      b+=1
      # print(df2['decision'][i],"= predicted truthful",)
# print("a=",a,"b=",b,"c=",c,"d=",d)
print((a+d)/320)

0.49375


**Extending Dictionary on 1600 Reviews**

In [ ]:
count= 0
reviews = [[i] for i in df2['text']]
print("Number of reviews=",len(reviews))

Number of reviews= 1600


In [ ]:
print(reviews[0])

["As a former Chicagoan, I'm appalled at the Amalfi Hotel Chicago. First of all, I was expecting luxury and hospitality, neither of which I received. There's an Experience Designer who is supposed to be like a 'personal concierge,' but my experience with my ED was terrible. I felt like he was trying to pressure me into staying more days than I wanted to. Not only that, but I couldn't understand what he was saying most of the time because he was talking so fast. When I finally got to my room, I was disappointed with the quality of the furniture and the room's cleanliness. I had to ask for a maid to come and give me clean towels because some of the towels in the bathroom were damp. On top of that, the bed was messily done; I could have done a better job on my own bed at home. I was angry at this point, because I was paying a lot of money for every night I was staying at Amalfi, and I didn't expect to be greeted with wet towels. I needed to use the Wi-Fi to download some important documen

In [ ]:
texts = []

for review in reviews:
  temp = re.sub(r'\[[0-9]*\]',' ',review[0])
  temp = re.sub(r'\s+',' ',temp)
  temp = temp.lower()
  temp = re.sub(r'\d',' ',temp)
  texts.append(re.sub(r'\s+',' ',temp)) 

In [ ]:
sentences=[]
for text in texts:
  sentence = nltk.sent_tokenize(text)
  sentences.append(sentence)

print(sentences[1599])

["as a former chicagoan, i'm appalled at the amalfi hotel chicago.", 'first of all, i was expecting luxury and hospitality, neither of which i received.', "there's an experience designer who is supposed to be like a 'personal concierge,' but my experience with my ed was terrible.", 'i felt like he was trying to pressure me into staying more days than i wanted to.', "not only that, but i couldn't understand what he was saying most of the time because he was talking so fast.", "when i finally got to my room, i was disappointed with the quality of the furniture and the room's cleanliness.", 'i had to ask for a maid to come and give me clean towels because some of the towels in the bathroom were damp.', 'on top of that, the bed was messily done; i could have done a better job on my own bed at home.', "i was angry at this point, because i was paying a lot of money for every night i was staying at amalfi, and i didn't expect to be greeted with wet towels.", 'i needed to use the wi-fi to down

In [ ]:
word_tokenized_reviews = []

for review in sentences:
  temp = [nltk.word_tokenize(sentence) for sentence in review]
  word_tokenized_reviews.append(temp)

print(word_tokenized_reviews[1599])

[['as', 'a', 'former', 'chicagoan', ',', 'i', "'m", 'appalled', 'at', 'the', 'amalfi', 'hotel', 'chicago', '.'], ['first', 'of', 'all', ',', 'i', 'was', 'expecting', 'luxury', 'and', 'hospitality', ',', 'neither', 'of', 'which', 'i', 'received', '.'], ['there', "'s", 'an', 'experience', 'designer', 'who', 'is', 'supposed', 'to', 'be', 'like', 'a', "'personal", 'concierge', ',', "'", 'but', 'my', 'experience', 'with', 'my', 'ed', 'was', 'terrible', '.'], ['i', 'felt', 'like', 'he', 'was', 'trying', 'to', 'pressure', 'me', 'into', 'staying', 'more', 'days', 'than', 'i', 'wanted', 'to', '.'], ['not', 'only', 'that', ',', 'but', 'i', 'could', "n't", 'understand', 'what', 'he', 'was', 'saying', 'most', 'of', 'the', 'time', 'because', 'he', 'was', 'talking', 'so', 'fast', '.'], ['when', 'i', 'finally', 'got', 'to', 'my', 'room', ',', 'i', 'was', 'disappointed', 'with', 'the', 'quality', 'of', 'the', 'furniture', 'and', 'the', 'room', "'s", 'cleanliness', '.'], ['i', 'had', 'to', 'ask', 'for'

In [ ]:
#Removing all the stopwords and the punctuations
for review in word_tokenized_reviews:   # the review will contain [[sentence],[sentence]]
  for i in range(len(review)):
    review[i] = [word for word in review[i] if word not in stopwords.words('english')]  #represents each sentence
    table = str.maketrans("","",string.punctuation)
    stripped = [w.translate(table) for w in review[i]]
    review[i] = list(filter(('').__ne__, stripped))
    print(review[i])

In [ ]:
# for each review in word_tokenized_reviews
count=0
additional_words=[]
for it in range(0,1600):#for each review
  print("Iteration="+str(it))
  selected_review = word_tokenized_reviews[it]  #selecting the review to be operated upon
  # print("Selected Review=",selected_review)
  candidate_spam_dictionary = []  #intermediate dictionary
  for i in seed[0]: #for each word in seed spam dictionary
    # print("Word selected ="+i)
    dissimilarity_weights = []  #for finding the dissimilarity between the word in seed spam dictionary and the words in the reviews
    try:
      v_spam = wiki2vec.get_word_vector(i)  #finding vector of each seed spam word
      # print("vspam=",v_spam)
    except Exception as e:
      # print("Spam Exception=",e)
      continue
    for j in range(len(selected_review)):  #for each sentence in selected review
      # print(selected_review[j])
      for k in selected_review[j]:         #for each word in the sentence
        # print(k)
        try:
          v_review = wiki2vec.get_word_vector(k)  #converting each word of the sentence into vector
        except Exception as e:
          continue
        # print("v_review=",v_review)
        dissim=distance.cosine(v_spam,v_review) #calculating the cosine distance
        # print(i,k,"=>",dissim)
        if(1-dissim>=0.5):  #similarity
          if [i,k,dissim] not in dissimilarity_weights:
            dissimilarity_weights.append([i,k,dissim])   #dissimilarity weights for each review word
    # print("Dissimilarity=",dissimilarity_weights)
    s = sorted(dissimilarity_weights, key = lambda x: (x[2]))   #sorting the dissimilarity weights of the words from seed words for the entire review
    # print("s=>",s)
    s_dash = []
    if len(s)>=15: #What if there are less than 15 words
      for j in range(0,15): #appending 15 most similar words to the s_dash
        s_dash.append(s[j])
    else:
      for j in range(len(s)):
        s_dash.append(s[j])
    # print("S_dash_length=",len(s_dash))
    # print("s_dash",s_dash)
    for j in range(len(s_dash)): #for each word in s_dash
      if(s_dash[j][1] not in seed[0]):# if s_dash is already present in the dictionary or similarity < 0.4 discard it 1-s_dash becoz its dissimilarity
        if s_dash[j][1] not in candidate_spam_dictionary:
          candidate_spam_dictionary.append(s_dash[j][1])
  # print("Candidate Spam dictionary:",candidate_spam_dictionary)  #No bug in Algorithm for Candidate spam dictionary

  for i in candidate_spam_dictionary:  #for each word in candidate spam dictionary
    # print("Candidate word selected=",i)
    dissimilarity_weights   = []
    v_candidate_dictionary  = wiki2vec.get_word_vector(i)  #vector of each candidate spam word because the candidate spam words has been extracted from the reviews
    for j in range(len(selected_review)):  #for each sentence in the selected review
      # print(selected_review[j])
      for k in selected_review[j]: #for each word in selected sentence
        try:
          v_review = wiki2vec.get_word_vector(k)  #converting each word of the sentence into vector
        except:
          continue
        dissim=distance.cosine(v_candidate_dictionary,v_review) #calculating the dissimilarity
        if [i,k,dissim] not in dissimilarity_weights: # What if the word is already there
          dissimilarity_weights.append([i,k,dissim])   #dissimilarity weights for each review word
    # print("dissimilarity_weights=",dissimilarity_weights)
    s = sorted(dissimilarity_weights, key = lambda x: (x[2]))  #sorting the dissimilarity weights of the words from seed words for the entire review
    # print("s=",s)
    s_double_dash = []
    if(len(s)>=15):
      for j in range(0,15): #Appending 15 most similar words
        s_double_dash.append(s[j][1]) # s_double_dash and s will be three dimensional array [k],storing only append 15 most similar words to s_dash
    else:
      for j in range(len(s)):
        s_double_dash.append(s[j][1])
    # print("s_double_dash=",s_double_dash) #printing the 15 words similar to the canidate spam dictionary
    if (len(set(s_double_dash) & set(seed[0]))>=3):#if there are 3 or more than 3 words in the seed dictionary then add the word of candidate spam dictionary to the seed spam dictionary
      # print("Len=",set(s_double_dash) & set(seed[0]))
      if i not in additional_words:
        # seed[0].append(i)
        additional_words.append(i)
        print("Word included=",i)
  # #       # system.exit(1)      


In [ ]:
print(len(additional_words))
print(additional_words) 

511
['beat', 'incredible', 'gourmand', 'big', 'good', 'huge', 'superb', 'first', 'honors', 'including', 'without', 'like', 'nicest', 'extremely', 'wished', 'comfortable', 'attractive', 'pleasant', 'reviews', 'enough', 'popular', 'amazing', 'perfect', 'made', 'room', 'helpful', 'easy', 'enjoyed', 'fun', 'close', 'prompt', 'beautiful', 'many', 'sure', 'negative', 'rewards', 'reflective', 'especially', 'daughter', 'loved', 'wonderfully', 'sparkling', 'music', 'quality', 'raved', 'nice', 'fresh', 'little', 'super', 'quick', 'spotless', 'book', 'biggest', 'kind', 'decent', 'pretty', 'bit', 'wardrobe', 'also', 'deep', 'one', 'well', 'breathtaking', 'lovely', 'sexy', 'handsome', 'choice', 'time', 'doubt', 'back', 'heavy', 'fabulous', 'yummy', 'spectacular', 'expensive', 'bad', 'sweet', 'common', 'star', 'greatest', 'unexpected', 'joy', 'anxious', 'nicely', 'wicked', 'watched', 'favourite', 'hardly', 'everyone', 'love', 'liked', 'yes', 'clean', 'friends', 'spacious', 'hotels', 'tastefully', 'n

In [ ]:
additional_words=[word for word in additional_words if word not in stopwords.words('english')]

In [ ]:
print(len(additional_words))
print(additional_words)


510
['beat', 'incredible', 'gourmand', 'big', 'good', 'huge', 'superb', 'first', 'honors', 'including', 'without', 'like', 'nicest', 'extremely', 'wished', 'comfortable', 'attractive', 'pleasant', 'reviews', 'enough', 'popular', 'amazing', 'perfect', 'made', 'room', 'helpful', 'easy', 'enjoyed', 'fun', 'close', 'prompt', 'beautiful', 'many', 'sure', 'negative', 'rewards', 'reflective', 'especially', 'daughter', 'loved', 'wonderfully', 'sparkling', 'music', 'quality', 'raved', 'nice', 'fresh', 'little', 'super', 'quick', 'spotless', 'book', 'biggest', 'kind', 'decent', 'pretty', 'bit', 'wardrobe', 'also', 'deep', 'one', 'well', 'breathtaking', 'lovely', 'sexy', 'handsome', 'choice', 'time', 'doubt', 'back', 'heavy', 'fabulous', 'yummy', 'spectacular', 'expensive', 'bad', 'sweet', 'common', 'star', 'greatest', 'unexpected', 'joy', 'anxious', 'nicely', 'wicked', 'watched', 'favourite', 'hardly', 'everyone', 'love', 'liked', 'yes', 'clean', 'friends', 'spacious', 'hotels', 'tastefully', 'n

In [ ]:
f= open("expandedSeed_on_1600_reviews.txt","w+")

In [ ]:
for i in range(len(additional_words)):
     f.write(additional_words[i]+"\r\n")
f.close()

**Counting Spam Words in Reviews and Exporting**

In [ ]:
from google.colab import files  #Upload the dictionary to find the word count
uploaded = files.upload()

Saving seed23.txt to seed23 (2).txt


In [ ]:
file_name = "seed23.txt"
uploaded[file_name].decode("utf-8")

'problem\r\npoor\r\nterrible\r\nunresponsive\r\ndiminished\r\nminiscule\r\ngem\r\nrude\r\noutrageous\r\nterrific\r\nunfriendly\r\nterrified\r\nscowled\r\nyelled\r\ndispleased\r\ntempered\r\nunhappy\r\ndirty\r\ndreary\r\naccused\r\nhumiliated\r\nvaguest\r\nridiculously\r\nugly\r\noverpriced\r\nrecession\r\nstressful\r\noutrageously\r\npoor\r\nmiserable\r\ncranky\r\nunappetizing\r\ndisappointed\r\nblindfolded\r\ndisgusted\r\nwarning\r\nsnubbed\r\ninept\r\nrepulsive\r\narrogant\r\ncallous\r\nawkward\r\nexcellent\r\nrelaxed\r\nwonderful\r\nheartbeat\r\nbest\r\nmassive\r\nelite\r\nclassy\r\nelegant\r\noutstanding\r\nmagnificent\r\nhomey\r\naffordable\r\nequipped\r\nextravagant\r\ncozy\r\nfriendly\r\nprofessional\r\ngreat\r\nfantastic\r\ndelicious\r\nincredibly\r\nbeauty\r\nrelieved\r\ncomplimentary\r\npriceless\r\ndefinitely\r\nrecommended\r\ngorgeous\r\nguarantees\r\ncelebrity\r\npleased\r\nexceptional\r\nsuggest'

In [ ]:
seed = uploaded[file_name].decode("utf-8").split("\r\n")
seed = [[i for i in seed]]
print(seed)

[['problem', 'poor', 'terrible', 'unresponsive', 'diminished', 'miniscule', 'gem', 'rude', 'outrageous', 'terrific', 'unfriendly', 'terrified', 'scowled', 'yelled', 'displeased', 'tempered', 'unhappy', 'dirty', 'dreary', 'accused', 'humiliated', 'vaguest', 'ridiculously', 'ugly', 'overpriced', 'recession', 'stressful', 'outrageously', 'poor', 'miserable', 'cranky', 'unappetizing', 'disappointed', 'blindfolded', 'disgusted', 'warning', 'snubbed', 'inept', 'repulsive', 'arrogant', 'callous', 'awkward', 'excellent', 'relaxed', 'wonderful', 'heartbeat', 'best', 'massive', 'elite', 'classy', 'elegant', 'outstanding', 'magnificent', 'homey', 'affordable', 'equipped', 'extravagant', 'cozy', 'friendly', 'professional', 'great', 'fantastic', 'delicious', 'incredibly', 'beauty', 'relieved', 'complimentary', 'priceless', 'definitely', 'recommended', 'gorgeous', 'guarantees', 'celebrity', 'pleased', 'exceptional', 'suggest']]


In [ ]:
count= 0
reviews = [[i] for i in df2['text']]
print("Number of reviews=",len(reviews))

Number of reviews= 1600


In [ ]:
print(reviews[0][0])

We stayed for a one night getaway with family on a thursday. Triple AAA rate of 173 was a steal. 7th floor room complete with 44in plasma TV bose stereo, voss and evian water, and gorgeous bathroom(no tub but was fine for us) Concierge was very helpful. You cannot beat this location... Only flaw was breakfast was pricey and service was very very slow(2hours for four kids and four adults on a friday morning) even though there were only two other tables in the restaurant. Food was very good so it was worth the wait. I would return in a heartbeat. A gem in chicago... 



In [ ]:
texts = []

for review in reviews:
  temp = re.sub(r'\[[0-9]*\]',' ',review[0])
  temp = re.sub(r'\s+',' ',temp)
  temp = temp.lower()
  temp = re.sub(r'\d',' ',temp)
  texts.append(re.sub(r'\s+',' ',temp)) 





#text will contain array of reviews in the string format!

In [ ]:
# sentences = nltk.sent_tokenize(texts[0])
# print(sentences)
#nltk.sent_tokenize() will tokenize the the sentences inside the reviews as string seperate strings

sentences=[]
for text in texts:
  sentence = nltk.sent_tokenize(text)
  sentences.append(sentence)

# for i in sentences:
#   print(i)

# sentences will contain the reviews which is tokenized as sentences with each review grouped as aray [[review],[review],[review]]

In [ ]:
word_tokenized_reviews = []

for review in sentences:
  temp = [nltk.word_tokenize(sentence) for sentence in review]
  word_tokenized_reviews.append(temp)

# for review in word_tokenized_reviews:
#   print(review)

# word_tokenized_review will contain = [[review],[review]]
# reviews will contain = [[sentence],[sentence]]
# sentence will contain = ['words','word','word']


In [ ]:
# for i in range(len(sentences)): #Removing the stop words for one review
#     sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

# print(sentences)

for review in word_tokenized_reviews:   # the review will contain [[sentence],[sentence]]
  for i in range(len(review)):
    review[i] = [word for word in review[i] if word not in stopwords.words('english')]

# for review in word_tokenized_reviews:  # All the stopwords is removed
#   print(review)


In [ ]:
#breaking each review in words
# print(word_tokenized_reviews[0])
review = []
for it in word_tokenized_reviews:
  words_in_each_review=[]
  for i in it: #for each sentence
    for j in i: #for each word
      words_in_each_review.append(j)
  review.append(words_in_each_review)

  

In [ ]:
print(review)

[['stayed', 'one', 'night', 'getaway', 'family', 'thursday', '.', 'triple', 'aaa', 'rate', 'steal', '.', 'th', 'floor', 'room', 'complete', 'plasma', 'tv', 'bose', 'stereo', ',', 'voss', 'evian', 'water', ',', 'gorgeous', 'bathroom', '(', 'tub', 'fine', 'us', ')', 'concierge', 'helpful', '.', 'beat', 'location', '...', 'flaw', 'breakfast', 'pricey', 'service', 'slow', '(', 'hours', 'four', 'kids', 'four', 'adults', 'friday', 'morning', ')', 'even', 'though', 'two', 'tables', 'restaurant', '.', 'food', 'good', 'worth', 'wait', '.', 'would', 'return', 'heartbeat', '.', 'gem', 'chicago', '...'], ['triple', 'rate', 'upgrade', 'view', 'room', 'less', '$', 'also', 'included', 'breakfast', 'vouchers', '.', 'great', 'view', 'river', ',', 'lake', ',', 'wrigley', 'bldg', '.', '&', 'tribune', 'bldg', '.', 'major', 'restaurants', ',', 'shopping', ',', 'sightseeing', 'attractions', 'within', 'walking', 'distance', '.', 'large', 'room', 'comfortable', 'bed', '.'], ['comes', 'little', 'late', "'m", '

In [ ]:
# print("seed[i]=",set(seed[0]))
# print("review[i]=",set(review[0]))

# print(set(review[0]) & set(seed[0]))
# print(len(set(review[0]) & set(seed[0])))

spam_word_count=[]
for i in review:
  # print("set(i)=",set(i))
  # print("set(review)=",set(i))
  print(len(set(i) & set(seed[0])))
  spam_word_count.append(len(set(i) & set(seed[0])))
  

In [ ]:
from pandas import DataFrame

In [ ]:
Label          =[]
hotel_name     =[]
polarity       =[]
source         =[]
review         =[]
# noun_proportion=[]
for Sr_no in range(1600):
  polarity.append(df2['polarity'][Sr_no])
  Label.append(df2['decision'][Sr_no])
  hotel_name.append(df2['hotel'][Sr_no])
  source.append(df2['source'][Sr_no])
  review.append(df2['text'][Sr_no])
  # noun_proportion.append(df2[''][Sr_no])

dataset = {'Label':Label,
             'Polarity':polarity,
             'Hotel':hotel_name,
             'Source':source,
             'Review':review,
             'Spam Word Count':spam_word_count
  }
df = DataFrame(dataset, columns= ['Label', 'Polarity','Hotel','Source','Review','Noun Proportion','Spam Word Count'])
df.to_excel (r'Spam_count_calculated.xlsx', index = None, header=True)


In [ ]:
import xlrd
import csv

def csv_from_excel():
    wb = xlrd.open_workbook('Spam_count_calculated.xlsx')
    sh = wb.sheet_by_name('Sheet1')
    your_csv_file = open('Spam_Count_updated.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

# runs the csv_from_excel function:
csv_from_excel()